@[TOC]

## 库

In [2]:
import os
import csv
import requests
import xlwt
import  re
import json
import time

### 配置

In [3]:
#根据个人浏览器信息进行修改
headers = {
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Mobile Safari/537.36'
    ,
    'Cookie': '_T_WM=67706607048; WEIBOCN_FROM=1110006030; ALF=1582777481; SCF=AqQddu0eGCw6Wh1xPsTyigWBFJH-P0ACsyLUFzNakys5tF6kBCjVpv4O6BDEGM4gShv5JHfuyjMoLBKfT5-Xwsc.; SUB=_2A25zK8jDDeRhGeNP41UT9yjIyj6IHXVQ1-iLrDV6PUJbktAKLUHSkW1NTk4PgJoxaitdQXaQL6znAIMdvJJs4-5l; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5q.Hx0pIs7PKpACzdnFYSZ5JpX5K-hUgL.Fo-p1hMES0qXeKz2dJLoIpUeBc8EdFH8SC-4BbHFSFH81F-RSF-4Sntt; SUHB=0qjEKc2Va_YMLH; SSOLoginState=1580185747; MLOGIN=1; XSRF-TOKEN=607e98; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D2304135671786192_-_WEIBO_SECOND_PROFILE_WEIBO%26fid%3D2304135671786192_-_WEIBO_SECOND_PROFILE_WEIBO%26uicode%3D10000011'
    #'ALF=1581501545; _T_WM=67706607048; H5_wentry=H5; backURL=https%3A%2F%2Fm.weibo.cn%2Fapi%2Fcomments%2Fshow%3Fid%3DIr5j4iRXW%26page%3D3; XSRF-TOKEN=11216a; WEIBOCN_FROM=1110006030; MLOGIN=1; SSOLoginState=1580006602; SCF=AqQddu0eGCw6Wh1xPsTyigWBFJH-P0ACsyLUFzNakys5zFt06rZeA1gEI0iP7HfWxZntbpMr8WTWhrxEdSVGB58.; SUB=_2A25zKIyaDeRhGeNP41UT9yjIyj6IHXVQ0hTSrDV6PUJbktAKLRL-kW1NTk4PgHLYgtoeuxFzuGDIDcybzoEoXvq9; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9W5q.Hx0pIs7PKpACzdnFYSZ5JpX5KzhUgL.Fo-p1hMES0qXeKz2dJLoIpUeBc8EdFH8SC-4BbHFSFH81F-RSF-4Sntt; SUHB=0IIlrfWMMkVsTI; M_WEIBOCN_PARAMS=uicode%3D20000174'
}

In [4]:
#文件保存地址
addrRoot='C:/Users/cascara/Desktop/seedcup/csv/blog/fans/'

In [5]:
#是否获取转发者具体个人信息
getConcreteInfoList=True#False#True#
isLogin=False#True#True

#是否登入采集个人信息

#无信息打印字符
infoNoExistStr='未知'


In [6]:
#是否处理微博文本内容
processText = True

### 构造表格，采集数据内容（修改这里获取想要的信息）

In [7]:
#获取个人具体信息范围、排列
infoRangeDict={
'性别':True,
'所在地':True,
    
'生日':False,
'家乡':False,
'公司':True,
'大学':True,
    
'昵称':False,
'简介':False,
'注册时间':False,
'阳光信用':False,
    
    #若无信息显示
'infoNoExist':'未知'
}



In [8]:
#获取博主信息范围、排列
userRangeDict={
'id':True,# 1323527941
'screen_name': True,#"Vista看天下"
    
'profile_image_url': False,#"https://tva2.sinaimg.cn/crop.0.0.180.180.180/4ee36f05jw1e8qgp5bmzyj2050050aa8.jpg?KID=imgbed,tva&Expires=1580290462&ssig=xPIoKDRR56"
'profile_url':False,# "https://m.weibo.cn/u/1323527941?uid=1323527941&luicode=10000011&lfid=1076031323527941"
'statuses_count': False,#微博数 77256
'verified': False,#true
'verified_type':False,# 3
'verified_type_ext': False,#0
'verified_reason': False,#"《Vista看天下》官方微博"
'close_blue_v': False,#false
    
'description': True,#"一个有趣的蓝V"
'gender': True,# "m"
    
'mbtype': False,#12
'urank': False,#48
'mbrank': False,#6
'follow_me':False,# false
'following':False,# false
    
'followers_count': True,#19657897
'follow_count': True,#1809
    
'cover_image_phone': False,#"https://tva1.sinaimg.cn/crop.0.0.640.640.640/549d0121tw1egm1kjly3jj20hs0hsq4f.jpg"
'avatar_hd': False,#"https://ww2.sinaimg.cn/orj480/4ee36f05jw1e8qgp5bmzyj2050050aa8.jpg"
'like': False,#false
'like_me': False,#false
'badge': False,#{enterprise: 1, gongyi_level: 1, bind_taobao: 1, dzwbqlx_2016: 1, follow_whitelist_video: 1,…}
    
#若无信息显示
'infoNoExist':'未知'
}


### 文件命名

In [9]:
def addrFile(tweeter,suffix):
    path=addrRoot+str(tweeter)+'/'
    if os.path.exists(path) is False:
         os.makedirs(path)
    address=path+tweeter+suffix+'.csv'
    return address  

### 生成信息标题

In [10]:
def getInfoTitle(Dict,prefix):
    titleList=[]
    for item in Dict:
        if(Dict.get(item) is True):
            titleList.append(prefix+item)
    return (titleList)

## 工具类，用来去除爬取的正文中一些不需要的链接、标签等

In [11]:
#工具类，用来去除爬取的正文中一些不需要的链接、标签等
class Tool:
    deleteImg = re.compile('<img.*?>')
    newLine =re.compile('<tr>|<div>|</tr>|</div>')
    deleteAite = re.compile('//.*?:')
    deleteAddr = re.compile('<a.*?>.*?</a>|<a href='+'\'https:')
    deleteTag = re.compile('<.*?>')
    deleteWord = re.compile('回复@|回覆@|回覆|回复')
 
    @classmethod
    def replace(cls,x):
        x = re.sub(cls.deleteWord,'',x)
        x = re.sub(cls.deleteImg,'',x)
        x = re.sub(cls.deleteAite,'',x)
        x = re.sub(cls.deleteAddr, '', x)
        x = re.sub(cls.newLine,'',x)
        x = re.sub(cls.deleteTag,'',x)
        return x.strip()

### 构造粉丝界面的url

In [12]:
###某微博账户的全部微博内容
def contentURL(id,pages):
    i=0
    urls=[]
    for page in pages:
        if page not in [0,1]:
            urls+=['https://m.weibo.cn/api/container/getIndex?containerid=231051_-_followers_-_'+str(id)+'&page='+str(page)]

    return urls

In [13]:
#将字典类型的信息格式传递为需要的信息列表
def getInfoList(infoDict,rangeDict):
    infoList=[]
    for item in rangeDict:
        if rangeDict.get(item) is True:
            content=infoDict.get(item)
            if content is not None:
                #处理微博文本内容  
                if item =='text':
                    if processText is True:
                        content=Tool.replace(content)
                infoList.append(content)      
            else:
                infoList.append(rangeDict['infoNoExist'])
    return infoList

### 统计所有的粉丝数

In [25]:
###在已有的一系列urls中进行操作
###筛选出微博转发内容进行操作
def reRatio(urls,csvWriter):
    notEnd= True
    
    fansUserTitle=getInfoTitle(userRangeDict,'')#粉丝信息标题
    infoTitle=getInfoTitle(infoRangeDict,'')#原文博主个人主页信息标题
    
    #写表格的标题
    if getConcreteInfoList is True:       
        csvWriter.writerow(fansUserTitle+infoTitle)        
    else:
        csvWriter.writerow(fansUserTitle)
        
    for url in urls:        
        
        response = requests.get(url,headers=headers)
        resjson = json.loads(response.text)    
        
        if resjson['ok'] ==0:
            print(url)
            notEnd=False
            break   
        
        cards=resjson['data']['cards']             
        
        if(len(cards)==1):
            try:        
                cards=cards[0]['card_group']
            except:
                print(url)
                print(cards)
                notEnd=False
                break     
            
        #遍历一个页面的所有微博    
        for card in cards:
            try:
                #fans个人信息
                 
                fansUserInfoDict=card['user']                                 
                infoList=[]                            


                #原文博主数据
                fansUserInfoList=getInfoList(fansUserInfoDict,userRangeDict)               
                infoList+=fansUserInfoList 


                fansUserID=fansUserInfoDict['id']

                #fansUserID为粉丝账号的ID                            
                #可在此对id进行信息采集                               

                if getConcreteInfoList is True:
                    infoDict=getInfo(isLogin,fansUserID)
                    otherInfoList=getInfoList(infoDict,infoRangeDict)      
                    infoList+=otherInfoList                          
                #print(infoList)
                #保存数据至csv
                csvWriter.writerow(infoList)                       

            #不断获取该博主对的影响力
            #break
            except:
                pass

        #延时，防止反爬
        time.sleep(3)
        
    return notEnd

### 获取个人主页中信息

In [15]:
def drillInfo(txt):
    keyInfo={}
    
    try:  
        resjson = json.loads(txt)  
        infodata = resjson.get('data')
        cards = infodata.get('cards')
        for l in range(0,len(cards)):
            temp = cards[l]
            card_group = temp.get('card_group')            
            #判断获取信息类型                     
            for card in card_group:                
                #将信息传入字典
                name=card.get('item_name')
                if name is not None:
                    content=card.get('item_content')
                    keyInfo[name]=content 
    except:
        pass
    return keyInfo

###  构建通过id访问个人主页的url

In [16]:
def infoUrl(id):
    url = "https://m.weibo.cn/api/container/getIndex?containerid=230283"+str(id)+"_-_INFO" 
    return url

## 爬取某id博主的个人信息

In [17]:
def getInfo(state,id):
    
    address=addrRoot+'info/'+str(state)+'id'+str(id)+'.txt'
    path=addrRoot+'info/'
    if os.path.exists(path) is False:
        os.makedirs(path)
    try:
        #已有文件
        if(os.path.exists(address)==True):
            fp = open(address,'r',encoding='utf-16')
            txt=fp.read()
            info=drillInfo(txt)
            fp.close()
        else:  
            fp = open(address,'w+',encoding='utf-16')
            url=infoUrl(id)
            if state is True:
                response = requests.get(url,headers=headers)
            else:
                response = requests.get(url)
            txt=response.text
            fp.write(response.text)        
            info=drillInfo(txt)
            fp.close()
    except:
        info=-1     
    
    return info

In [18]:
def getExatInfo(item,state,id):
    info=getInfo(state,id)
    content=info.get(item)
    if content is not None:
        return content
    else:
        return infoNoExistStr        

In [19]:
###  构造热门界面访问

In [20]:
def downloadData(id):
    tweeter=getExatInfo('昵称',2,int(id))
    batch=0
    while(1):

        fileAddr=addrFile(tweeter,'batch'+str(batch))
        if os.path.exists(fileAddr) is True:
            print(fileAddr+'已存在，跳过采集')                
        else:
            print('文件将写入：'+fileAddr)
            fp = open(fileAddr,'w+',newline='',encoding='utf-16')
            writer=csv.writer(fp)
            if reRatio(contentURL(id,range(20*batch,20*(batch+1))),writer) is False:
                fp.close()
                break

            fp.close()
            print('第'+str(batch)+'批数据已记录完毕')
        batch+=1

In [27]:
id=input('博主id：')

downloadData(id)

博主id： 1574684061


文件将写入：C:/Users/cascara/Desktop/seedcup/csv/blog/fans/陈赫/陈赫batch0.csv
https://m.weibo.cn/api/container/getIndex?containerid=231051_-_followers_-_1574684061&page=11
